In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

Classification function

In [31]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.metrics import accuracy_score



In [32]:
def class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = None,
x_train = None,
x_test =  None,
y_train = None,
y_test =  None,
params = None,
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring = 'roc_auc',
logistic_reg = None,
tree_based = None):
    ###############################################################################
    
   
    features = list(x_train.columns)

    ss = StandardScaler()
    x_train = pd.DataFrame(ss.fit_transform(x_train), columns = features)
    x_test = pd.DataFrame(ss.fit_transform(x_test), columns = features)

    skfold = StratifiedKFold(n_splits= splits, shuffle=True, random_state=0)

    rs = RandomizedSearchCV(model, params, cv=skfold,  return_train_score=True, refit=True, scoring=scoring)
    rs.fit(x_train,y_train)
    rs.best_params_

    class_dict = {}
    model= rs.best_estimator_
    class_dict['mod'] = model
    ###################################    

    print('best params: ',rs.best_params_)
    class_dict['best params'] = rs.best_params_

    print('score: ',rs.score(x_train,y_train))
    class_dict['score'] = rs.score(x_train,y_train)

    print('  ')
    print('test score: ',rs.score(x_test,y_test))
    class_dict['test score'] = rs.score(x_test,y_test)

    from sklearn.metrics import confusion_matrix

    train_pred = rs.best_estimator_.predict(x_train)

    print('confusion matrix train')
    print(confusion_matrix(y_train, train_pred))
    
    class_dict['confusion_matrix_train'] = confusion_matrix(y_train, train_pred)
    print(' ')
    
    test_pred = rs.best_estimator_.predict(x_test)
    print('confusion matrix test')
    print(confusion_matrix(y_test, test_pred))
    class_dict['confusion_matrix_test'] = confusion_matrix(y_test, test_pred)
    print(' ')
    


   
    print(' ')
    
    print("train roc_auc_score: ", roc_auc_score(y_train, rs.predict_proba(x_train)[:,1]))
    
    class_dict['train_roc_auc_score'] = roc_auc_score(y_train, rs.predict_proba(x_train)[:,1])
    class_dict['predicted_probabilities_1_train']  = rs.predict_proba(x_train)[:,1]
    
    print(' ')
    
    print("test roc_auc_score: ", roc_auc_score(y_test, rs.predict_proba(x_test)[:,1]))
    
    class_dict['roc_auc_score'] = roc_auc_score(y_test, rs.predict_proba(x_test)[:,1])

    class_dict['predicted_probabilities_1_test'] = rs.predict_proba(x_test)[:,1]
    
    print(' ')
    #calculate KS train_______________________________________________________________________________________________
    
    predprob =list(class_dict['predicted_probabilities_1_train'])

    y = list(y_train)

    data = {'target_train':y,'predprob':predprob}

    df_prob_train= pd.DataFrame(data)
    df_prob_train.head()

    from scipy.stats import ks_2samp
    
    print('Training KS statistic')
    train_ks = ks_2samp(df_prob_train.loc[df_prob_train.target_train==0,"predprob"], df_prob_train.loc[df_prob_train.target_train==1,"predprob"])
    print(train_ks)
    class_dict['train_ks'] = train_ks
    
    print(' ')
    #calculate KS test_______________________________________________________________________________________________
    predprob2 =list(class_dict['predicted_probabilities_1_test'])

    y_ = list(y_test)

    data = {'target_test':y_,'predprob':predprob2}

    df_prob_test= pd.DataFrame(data)
    df_prob_test.head()

    
    print('Test KS statistic')
    test_ks = ks_2samp(df_prob_test.loc[df_prob_test.target_test==0,"predprob"], df_prob_test.loc[df_prob_test.target_test==1,"predprob"])
    print(test_ks)
    class_dict['test_ks'] = test_ks
    print(' ')
    #end calculate KS test_______________________________________________________________________________________________
   

    if logistic_reg == True:

        coefs = pd.Series(model.coef_[0], name = 'coef' )

        varnames = pd.Series(features, name = 'features')


        pd.set_option('display.max_rows', None) # or 1000.

        abs_coefs = pd.Series(np.abs(coefs),name = 'abs_val')

        coefs_ = pd.concat([varnames,coefs, abs_coefs], axis=1)

        # #return:
        model_importances = coefs_.sort_values(by=['abs_val'], ascending=False)
        model_importances.index=(range(model_importances.shape[0]))
        model_importances
        class_dict['model_importances'] = model_importances

        print(model_importances)
        print(' ')


        print('model_importances.shape: ',model_importances.shape)

    elif tree_based == True:
        pd.set_option('display.max_rows', None) # or 1000.

        #return
        varnames = pd.Series(features, name = 'features')

        importances_ = pd.Series(model.feature_importances_, name = 'importances')

        df = pd.concat([varnames,importances_], axis=1)

        model_importances = df.sort_values(by=['importances'], ascending=False)

        model_importances.index=(range(model_importances.shape[0]))

        class_dict['model_importances'] = model_importances

        print(model_importances)



    return class_dict

label_encode function

In [17]:
def label_encode(df,feature):
    from sklearn.preprocessing import LabelEncoder
    lencoder = LabelEncoder()
    df[feature] = lencoder.fit_transform(df[feature]).reshape((-1,1))

### Import data

In [39]:
X_train_dep = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\X_train_dep.csv')

X_test_dep = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\X_test_dep.csv')

y_train_dep_df = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\y_train_dep_df.csv')

y_test_dep_df  = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\y_test_dep_df.csv')

In [10]:
select_dtypes(exclude=['int64','float64','bool'])

,job,marital,education,default,housing,loan,contact,month
0,admin,single,secondary,no,yes,no,telephone,may
1,services,single,secondary,no,yes,no,cellular,nov
2,blue-collar,married,primary,no,yes,no,cellular,may
3,management,married,secondary,no,yes,no,cellular,nov
4,blue-collar,married,secondary,no,yes,no,cellular,jul
...,...,...,...,...,...,...,...,...
27995,blue-collar,married,primary,no,yes,yes,unknown,may
27996,management,married,tertiary,no,no,no,cellular,nov
27997,blue-collar,single,primary,no,yes,no,cellular,apr
27998,admin,single,secondary,no,yes,no,unknown,may


In [13]:
char_cols_lst = list(X_train_dep.select_dtypes(exclude=['int64','float64','bool']).columns)
char_cols_lst

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month']

In [8]:
X_train_dep

,Unnamed: 0,age,job,marital,education,default,balance,housing,loan,contact,...,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct
0,39336,30,admin,single,secondary,no,2,yes,no,telephone,...,0,0,0,0,0,0,0,1,0,0
1,25184,36,services,single,secondary,no,1482,yes,no,cellular,...,0,0,0,0,0,0,0,0,1,0
2,34484,35,blue-collar,married,primary,no,147,yes,no,cellular,...,0,0,0,0,0,0,0,1,0,0
3,25237,33,management,married,secondary,no,480,yes,no,cellular,...,0,0,0,0,0,0,0,0,1,0
4,16170,46,blue-collar,married,secondary,no,209,yes,no,cellular,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,2072,31,blue-collar,married,primary,no,0,yes,yes,unknown,...,0,0,0,0,0,0,0,1,0,0
27996,27318,39,management,married,tertiary,no,20932,no,no,cellular,...,0,0,0,0,0,0,0,0,1,0
27997,32879,33,blue-collar,single,primary,no,557,yes,no,cellular,...,0,0,0,0,0,0,0,0,0,0
27998,2577,31,admin,single,secondary,no,513,yes,no,unknown,...,0,0,0,0,0,0,0,1,0,0


### Label Encode

In [18]:
for categ_feat in char_cols_lst:
    label_encode(X_train_dep,categ_feat)

In [19]:
X_train_dep[char_cols_lst].head()

,job,marital,education,default,housing,loan,contact,month
0,0,2,1,0,1,0,1,8
1,7,2,1,0,1,0,0,9
2,1,1,0,0,1,0,0,8
3,4,1,1,0,1,0,0,9
4,1,1,1,0,1,0,0,5


In [20]:
for categ_feat in char_cols_lst:
    label_encode(X_test_dep,categ_feat)

### drop dummies 

In [21]:
dummies_lst = ['job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_married',
 'marital_single',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'default_yes',
 'housing_yes',
 'loan_yes',
 'contact_telephone',
 'contact_unknown',
 'month_aug',
 'month_dec',
 'month_feb',
 'month_jan',
 'month_jul',
 'month_jun',
 'month_mar',
 'month_may',
 'month_nov',
 'month_oct']

In [23]:
X_train_dep_dd = X_train_dep.drop(columns = dummies_lst)

In [35]:
X_train_dep_dd.shape

(28000, 14)

In [37]:
X_train_dep_dd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  28000 non-null  int64
 1   age         28000 non-null  int64
 2   job         28000 non-null  int32
 3   marital     28000 non-null  int32
 4   education   28000 non-null  int32
 5   default     28000 non-null  int32
 6   balance     28000 non-null  int64
 7   housing     28000 non-null  int32
 8   loan        28000 non-null  int32
 9   contact     28000 non-null  int32
 10  day         28000 non-null  int64
 11  month       28000 non-null  int32
 12  duration    28000 non-null  int64
 13  campaign    28000 non-null  int64
dtypes: int32(8), int64(6)
memory usage: 2.1 MB


In [36]:
X_test_dep_dd = X_test_dep.drop(columns = dummies_lst)
X_test_dep_dd.shape

(12000, 14)

In [38]:
X_test_dep_dd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  12000 non-null  int64
 1   age         12000 non-null  int64
 2   job         12000 non-null  int32
 3   marital     12000 non-null  int32
 4   education   12000 non-null  int32
 5   default     12000 non-null  int32
 6   balance     12000 non-null  int64
 7   housing     12000 non-null  int32
 8   loan        12000 non-null  int32
 9   contact     12000 non-null  int32
 10  day         12000 non-null  int64
 11  month       12000 non-null  int32
 12  duration    12000 non-null  int64
 13  campaign    12000 non-null  int64
dtypes: int32(8), int64(6)
memory usage: 937.6 KB


In [41]:
y_train_dep_df.head()

,Unnamed: 0,y_label
0,39336,False
1,25184,False
2,34484,False
3,25237,False
4,16170,False


In [42]:
y_test_dep_df.head()

,Unnamed: 0,y_label
0,13130,False
1,11127,True
2,17899,True
3,39652,False
4,25890,False


In [43]:
mod_1 = class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = GradientBoostingClassifier(),
x_train = X_train_dep_dd,
x_test =  X_test_dep_dd,
y_train = y_train_dep_df.y_label,
y_test =  y_test_dep.y_label,
params ={'max_depth':[1,2],'n_estimators':[1000, 2000, 3000, 4000, 5000], 'learning_rate':[0.01,0.001],
         'random_state':[0]},
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring= "accuracy",
logistic_reg = False,
tree_based = True)

best params:  {'random_state': 0, 'n_estimators': 4000, 'max_depth': 2, 'learning_rate': 0.01}
score:  0.9465357142857143
  
test score:  0.93775
confusion matrix train
[[25617   356]
 [ 1141   886]]
 
confusion matrix test
[[10907   224]
 [  523   346]]
 
 
train roc_auc_score:  0.9581473748183453
 
test roc_auc_score:  0.9425778202242383
 
Training KS statistic
KstestResult(statistic=0.8021075774278974, pvalue=0.0)
 
Test KS statistic
KstestResult(statistic=0.7538659539355509, pvalue=0.0)
 
      features  importances
0     duration      0.64273
1   Unnamed: 0      0.22753
2        month      0.05490
3      housing      0.02950
4          age      0.02265
5          day      0.00924
6      balance      0.00420
7      contact      0.00260
8     campaign      0.00255
9    education      0.00234
10     marital      0.00100
11         job      0.00066
12        loan      0.00007
13     default      0.00003
